In [1]:
import os
if os.getcwd().endswith('/notebook'):
    os.chdir('..')

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from model import DynamicDataset, normal_func, Cruys2014
from evaluation import compute_frequency_band_mapping
import seaborn as sns

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
cruys_2hidden = Cruys2014(num_hidden_layers=2)
cruys_3hidden = Cruys2014(num_hidden_layers=3)
cruys_4hidden = Cruys2014(num_hidden_layers=4)

In [5]:
cruys_4hidden.summary()

=== Single model ===
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 3, 50)             650000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 150)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               

In [6]:
train_ds = np.load('output/ukwac-train.npy')
dev_ds = np.load('output/ukwac-dev.npy')

In [7]:
train_ds_dynamic = DynamicDataset(train_ds)
dev_ds_dynamic = DynamicDataset(dev_ds)

def train_epoch(model):
    log = model.fit(*train_ds_dynamic.generate(), 
                         validation_data=dev_ds_dynamic.generate(),
                         batch_size=1000, epochs=1, verbose=1)
    return log.history['loss'][0], log.history['val_loss'][0]

In [ ]:
%%time
# "we use mini-batch L-BFGS (Liu and Nocedal, 1989) with 1000 pairs of good and corrupt tuples 
# per batch for training, and train for 10 epochs" (van der Cruys, 2014)
# --> tf.keras doesn't have an implementation of L-BFGS and we couldn't get good results with
# 10 epochs of training so we use a different settings. However, I can't imagine that it makes
# so much a difference.
log = [train_epoch(cruys_2hidden) for _ in range(10)]

Train on 6569639 samples, validate on 729960 samples
Epoch 1/1
6569639/6569639 [==============================]6569639/6569639 [==============================] - 224s 34us/step - loss: 1.8430 - contrastive_accuracy: 0.5096 - val_loss: 1.4210 - val_contrastive_accuracy: 0.6055

Train on 6569639 samples, validate on 729960 samples
Epoch 1/1
 358000/6569639 [>.............................] 358000/6569639 [>.............................] - ETA: 3:56 - loss: 1.3844 - contrastive_accuracy: 0.6139

In [ ]:
%%time
log = [train_epoch(cruys_3hidden) for _ in range(10)]

In [ ]:
%%time
log = [train_epoch(cruys_4hidden) for _ in range(10)]